# EDA complète — Nantes Métropole (2012–2022)
Ce notebook réalise les contrôles de qualité, l'exploration des données et quelques visualisations clés sur `data/processed_csv/master_ml.csv`.
> Exécuter après l'ETL : `docker compose run --rm app src/etl/build_master.py`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
DATA = Path('data/processed_csv/master_ml.csv')
df = pd.read_csv(DATA)
print(df.shape)
df.head()

## 1) Contrôles qualité (QA)

In [ ]:
# unicité de la clé (si présentes)
key_cols = [c for c in ['code_commune_insee','type_scrutin','annee','tour'] if c in df.columns]
if key_cols:
    dup = df.duplicated(subset=key_cols).sum()
    print('Doublons (clé):', dup)
else:
    print('Colonnes clés manquantes, QA limité.')

# complétude colonnes clés
for c in key_cols:
    miss = df[c].isna().mean()*100
    print(f'Manquants {c}: {miss:.1f}%')

# cohérence pourcentages si colonnes présentes
for pct_col in ['turnout_pct','blancs_pct','nuls_pct']:
    if pct_col in df.columns:
        print(pct_col, 'min/max:', df[pct_col].min(), df[pct_col].max())

# somme des voix_% (si colonnes voix_pct_*)
voix_pct_cols = [c for c in df.columns if c.startswith('voix_pct_')]
if voix_pct_cols:
    s = df[voix_pct_cols].sum(axis=1)
    print('Somme moyenne voix_pct_*:', s.mean(), 'écart-type:', s.std())

## 2) Vue d'ensemble

In [ ]:
print('Années ->', sorted(df['annee'].dropna().unique().tolist()) )
if 'type_scrutin' in df.columns:
    print('Scrutins ->', df['type_scrutin'].value_counts().to_dict())
if 'parti_en_tete' in df.columns:
    print('Répartition parti_en_tete ->', df['parti_en_tete'].value_counts(normalize=True).round(3).to_dict())

## 3) Tendances de participation (turnout / blancs / nuls)

In [ ]:
group_cols = [c for c in ['annee','type_scrutin'] if c in df.columns]
agg = {}
for c in ['turnout_pct','blancs_pct','nuls_pct']:
    if c in df.columns:
        agg[c] = 'mean'
if group_cols and agg:
    trend = df.groupby(group_cols).agg(agg).reset_index()
    display(trend.head())
    # Plot (un graphe par métrique)
    if 'turnout_pct' in trend.columns:
        plt.figure()
        for k, sub in trend.groupby('type_scrutin') if 'type_scrutin' in trend.columns else [(None, trend)]:
            plt.plot(sub['annee'], sub['turnout_pct'], marker='o', label=str(k) if k is not None else None)
        plt.title('Taux de participation (moyenne par scrutin et année)')
        plt.xlabel('Année'); plt.ylabel('turnout_pct')
        if 'type_scrutin' in trend.columns: plt.legend()
        plt.show()
    if 'blancs_pct' in trend.columns:
        plt.figure()
        for k, sub in trend.groupby('type_scrutin') if 'type_scrutin' in trend.columns else [(None, trend)]:
            plt.plot(sub['annee'], sub['blancs_pct'], marker='o', label=str(k) if k is not None else None)
        plt.title('Blancs (moyenne)'); plt.xlabel('Année'); plt.ylabel('blancs_pct')
        if 'type_scrutin' in trend.columns: plt.legend()
        plt.show()
    if 'nuls_pct' in trend.columns:
        plt.figure()
        for k, sub in trend.groupby('type_scrutin') if 'type_scrutin' in trend.columns else [(None, trend)]:
            plt.plot(sub['annee'], sub['nuls_pct'], marker='o', label=str(k) if k is not None else None)
        plt.title('Nuls (moyenne)'); plt.xlabel('Année'); plt.ylabel('nuls_pct')
        if 'type_scrutin' in trend.columns: plt.legend()
        plt.show()

## 4) Corrélations (indicateurs ↔ résultats)

In [ ]:
# Sélection simple : indicateurs + parts de vote si présentes
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
subset_cols = []
for c in ['population','median_income','unemployment_rate','poverty_rate','security_incidents_per_1000',
          'turnout_pct','blancs_pct','nuls_pct']:
    if c in df.columns: subset_cols.append(c)
subset_cols += [c for c in df.columns if c.startswith('voix_pct_')][:8]
subset_cols = [c for c in subset_cols if c in num_cols]
if subset_cols:
    corr = df[subset_cols].corr()
    import numpy as np
    plt.figure()
    plt.imshow(corr.values)
    plt.title('Corrélations (sous-ensemble)')
    plt.xticks(range(len(subset_cols)), subset_cols, rotation=90)
    plt.yticks(range(len(subset_cols)), subset_cols)
    for i in range(corr.shape[0]):
        for j in range(corr.shape[1]):
            val = corr.values[i, j]
            plt.text(j, i, f"{val:.2f}", ha='center', va='center')
    plt.tight_layout(); plt.show()

## 5) Importance des variables (Random Forest rapide)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df2 = df.copy()
df2 = df2.dropna(subset=['parti_en_tete']) if 'parti_en_tete' in df2.columns else df2
y = df2['parti_en_tete'].astype(str) if 'parti_en_tete' in df2.columns else None
categorical = [c for c in ['type_scrutin','tour'] if c in df2.columns]
exclude = set(['code_commune_insee','nom_commune','code_epci','date_scrutin','parti_en_tete','winner_prev','estime'])
numeric = [c for c in df2.columns if c not in exclude and c not in categorical and pd.api.types.is_numeric_dtype(df2[c])]
if y is not None and len(y)>0:
    X = df2[numeric + categorical]
    pre = ColumnTransformer([
        ('num', StandardScaler(with_mean=False), numeric),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
    ], remainder='drop')
    rf = Pipeline([('prep', pre), ('clf', RandomForestClassifier(n_estimators=300, random_state=42))])
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    rf.fit(Xtr, ytr)
    print('Accuracy RF holdout:', accuracy_score(yte, rf.predict(Xte)))
else:
    print('parti_en_tete indisponible — importance non calculée.')

## 6) Idées de cartes (à exécuter si GeoJSON disponible)

In [ ]:
# Exemple d'esquisse (nécessite un GeoJSON des communes) :
# import json
# import matplotlib.pyplot as plt
# import shapely.geometry as geom
# import shapely.ops as ops
# -> Rejoindre le GeoJSON des communes (INSEE) avec df sur code_commune_insee,
#    puis colorier par parti_en_tete.
print('Ajoutez un GeoJSON des communes pour produire une carte thématique.')

## 7) Carte — parti en tête par commune (exemple)
Exécuter d'abord : `docker compose run --rm app python src/etl/fetch_geojson.py` pour récupérer le GeoJSON.
Ensuite, choisissez une année et un type de scrutin (ex. 2022, présidentielles 1er tour).

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection
from pathlib import Path

GEO = Path('data/geo/communes_nantes_metropole.geojson')
if not GEO.exists():
    raise SystemExit('GeoJSON manquant. Lancez: docker compose run --rm app python src/etl/fetch_geojson.py')

with open(GEO, 'r', encoding='utf-8') as f:
    gj = json.load(f)

# Choix
YEAR = int(sorted(df['annee'].dropna().unique())[-1]) if 'annee' in df.columns else 2022
SCRUTIN = 'presidentielle'  # adapter au besoin
TOUR = 1

# Prépare les données (vainqueur par commune)
key_cols = [c for c in ['code_commune_insee','annee','type_scrutin','tour','parti_en_tete'] if c in df.columns]
subset = df[key_cols].dropna()
subset = subset[(subset['annee']==YEAR) & (subset['type_scrutin'].str.contains(SCRUTIN, case=False, na=False))]
if 'tour' in subset.columns:
    subset = subset[subset['tour']==TOUR]
winners = subset.set_index('code_commune_insee')['parti_en_tete'].to_dict()

partis = sorted(set(winners.values()))
parti_to_id = {p:i for i,p in enumerate(partis)}

patches = []
vals = []
for ft in gj.get('features', []):
    props = ft.get('properties', {})
    code = str(props.get('code', '')).zfill(5)
    geom = ft.get('geometry', {})
    if not code or not geom:
        continue
    polys = []
    if geom.get('type') == 'Polygon':
        polys = [geom['coordinates']]
    elif geom.get('type') == 'MultiPolygon':
        polys = geom['coordinates']
    pid = parti_to_id.get(winners.get(code, None), None)
    for poly in polys:
        if not poly:
            continue
        ring = poly[0]
        patches.append(MplPolygon(ring, True))
        vals.append(pid if pid is not None else np.nan)

fig, ax = plt.subplots(figsize=(8,8))
pc = PatchCollection(patches, alpha=0.85)
# map numeric ids to colors using default colormap
if len(vals):
    arr = np.array([v if v==v else -1 for v in vals])
    # Normalize to [0,1] ignoring -1
    if np.any(arr>=0):
        vmax = max(arr[arr>=0]) or 1
        colors = np.where(arr>=0, (arr / max(vmax,1)), 0.0)
        pc.set_array(colors)
ax.add_collection(pc)
ax.autoscale_view()
ax.set_aspect('equal', 'box')
ax.set_title(f'Parti en tête — {YEAR}, {SCRUTIN} T{TOUR}')
plt.show()


---
### Fin de l’EDA